# Algoritmos de Otimização

No Deep Learning temos como propósito que nossas redes neurais aprendam a aproximar uma função de interesse, como o preço de casas numa regressão, ou a função que classifica objetos numa foto, no caso da classificação.

Esse aprendizado se dá por meio da otimizição da rede neural para minimizar a função de custo, mas podemos usar vários algoritmos diferentes para alcançar esse mínimo. Já vimos anteriormente o gradiente descendente, mas como ele é muito importante e serve de base para os outros métodos, vamos começar recapitulando ele.

## Descida de Gradiente Estocástica (SGD)
Na descida de gradiente estocástica separamos os nossos dados de treino em vários subconjuntos, que chamamos de mini-batches. No começo eles serão pequenos, como 32-128 exemplos, para aplicações mais avançadas eles tendem a ser muito maiores, na ordem de 1024 e até mesmo 8192 exemplos por mini-batch.

Como na descida de gradiente padrão, computamos o gradiente da função de custo em relação aos exemplos, e subtraímos o gradiente vezes uma taxa de apredizado dos parâmetros da rede. Podemos ver o SGD como tomando um passo pequeno na direção de maior redução do valor da loss.

### Equação
$w_{t+1} = w_t - \eta \cdot \nabla L$

### Código

In [1]:
import jax
def sgd(weights, gradients, eta):
    return jax.tree_util.tree_multimap(lambda w, g: w - eta*g, weights, gradients)

# IMAGEM LEGAL DE SGD

## Momentum
Um problema com o uso de mini-batches é que agora estamos **estimando** a direção que diminui a função de perda no conjunto de treino, e quão menor o mini-batch mais ruidosa é a nossa estimativa. Para consertar esse problema do ruído nós introduzimos a noção de momentum. O momentm faz sua otimização agir como uma bola pesada descendo uma montanha, então mesmo que o caminho seja cheio de montes e buracos a direção da bola não é muito afetada. De um ponto de vista mais matemático as nossas atualizações dos pesos vão ser uma combinação entre os gradientes desse passo e os gradientes anteriores, estabilizando o treino.

### Equação
$v_{t} = \gamma v_{t-1} + \nabla L \quad \text{o gamma serve como um coeficiente ponderando entre usar os updates anteriores e o novo gradiente} \\
w_{t+1} = w_t - \eta v_t
$

### Código

In [2]:
def momentum(weights, gradients, eta, mom, gamma):
    mom = jax.tree_util.tree_multimap(lambda v, g: gamma*v + g, weights, mom)
    return jax.tree_util.tree_multimap(lambda w, v: w - eta*mom, weights, mom)

## RMSProp
Criado por Geoffrey Hinton durante uma aula, esse método é o primeiro **método adaptivo** que estamos vendo. O que isso quer dizer é que o método tenta automaticamente computar uma taxa de aprendizado diferente para cada um dos pesos da nossa rede neural, usando taxas pequenas para parâmetros que sofrem atualização frequentemente e taxas maiores para parâmetros que são atualizados mais raramente, permitindo uma otimização mais rápida. 

Mais especificamente, o RMSProp divide o update normal do SGD pela raiz da soma dos quadrados dos gradientes anteriores (por isso seu nome Root-Mean-Square Proportional), assim reduzindo a magnitude da atualização de acordo com as magnitudes anteriores.

### Equação
$
\nu_{t} = \gamma \nu_{t-1} + (1 - \gamma) (\nabla L)^2 \\
w_{t+1} = w_t - \frac{\eta \nabla L}{\sqrt{\nu_t + \epsilon}} 
$

### Código

In [3]:
def computa_momento(updates, moments, decay, order):
    return jax.tree_multimap(
      lambda g, t: (1 - decay) * (g ** order) + decay * t, updates, moments)
def rmsprop(updates, nu, gamma):
    nu = computa_momento(updates, nu, gamma, 2)
    updates = jax.tree_multimap(
        lambda g, n: g * jax.lax.rsqrt(n + eps), updates, nu)
    return updates, nu

## Adam
Por fim o Adam usa ideias semelhantes ao Momentum e ao RMSProp, mantendo médias exponenciais tanto dos gradientes passados, quanto dos seus quadrados.

### Equação
$
m_t = \beta_1 m_{t-1} + (1 - \beta_1) \nabla L \\
v_t = \beta_2 v_{t-1} + (1 - \beta_2) (\nabla L)^2 \\
w_{t+1} = w_t - \frac{\eta m_t}{\sqrt{v_t} \epsilon} 
$

### Código

In [4]:
def adam(updates, mu, nu, b1, b2):
    mu = computa_momento(updates, mu, b1, 1)
    nu = computa_momento(updates, nu, b2, 2)
    updates = jax.tree_multimap(
        lambda m, v: m / (jnp.sqrt(v + eps_root) + eps), mu, nu)
    return updates, mu, nu